# Data Gathering
## Import Libraries & Tools

In [1]:
import pandas as pd
import numpy as np
import time
import glob
import re
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from datetime import date

## Create Directory Structure

In [2]:
DATA_DIR = os.path.join(
    os.path.dirname(os.path.realpath("__file__")), "data"
)

BOULDER_MEN_DIR            = os.path.join(DATA_DIR, "Boulder/Men")
BOULDER_WOMEN_DIR          = os.path.join(DATA_DIR, "Boulder/Women")
LEAD_MEN_DIR               = os.path.join(DATA_DIR, "Lead/Men")
LEAD_WOMEN_DIR             = os.path.join(DATA_DIR, "Lead/Women")
SPEED_MEN_DIR              = os.path.join(DATA_DIR, "Speed/Men")
SPEED_WOMEN_DIR            = os.path.join(DATA_DIR, "Speed/Women")
COMBINED_MEN_DIR           = os.path.join(DATA_DIR, "Combined/Men")
COMBINED_WOMEN_DIR         = os.path.join(DATA_DIR, "Combined/Women")
BOULDER_AND_LEAD_MEN_DIR   = os.path.join(DATA_DIR, "Boulder & Lead/Men")
BOULDER_AND_LEAD_WOMEN_DIR = os.path.join(DATA_DIR, "Boulder & Lead/Women")

dirs = [BOULDER_MEN_DIR, BOULDER_WOMEN_DIR, LEAD_MEN_DIR, LEAD_WOMEN_DIR,
       SPEED_MEN_DIR, SPEED_WOMEN_DIR, COMBINED_MEN_DIR, COMBINED_WOMEN_DIR,
       BOULDER_AND_LEAD_MEN_DIR, BOULDER_AND_LEAD_WOMEN_DIR]

# Create directory if it doesn't exist
for dir in dirs:
    if not os.path.exists(dir):
        os.makedirs(dir)
        
# File to store names of events that have already been scraped
try:
    ALREADY_SCRAPED = os.path.join(DATA_DIR, "scraped_events.txt")
    # Create file
    with open(ALREADY_SCRAPED, 'x') as fp:
        pass
except:
    if os.stat(ALREADY_SCRAPED).st_size == 0:
        print("No data has been scraped yet!")

## IFSCScraper Class Definition

In [9]:
class IFSCScraper():
    """
    Define a class for the scraper that will be used to gather data from the IFSC website
    (ifsc-climbing.org)
    Includes methods that allow for scraping different pages and different information
    """
    # Page url
    url  = 'https://www.ifsc-climbing.org/index.php/world-competition/last-result'
    url2 = 'https://ifsc.results.info/#/athlete/'
    
    def __init__(self, debug=False):
        """
        Initialize a scraper object with its own browser instance
        Input:
            debug - Indicates whether this is a debug instance for quicker development
        """
        self.debug = debug
        self.year_events = {}

        self.generate_driver()            
        time.sleep(1)
    
    def generate_driver(self):
        """
        Initialize Selenium web browser
        Input:
            N/A
        """
        try:
            self.driver = webdriver.Firefox()
        except:
            print('Error: Could not create WebDriver object...')
        
    def load_page(self, link, athlete_page=0, timeout=10, wait_after=1):
        """
        Helper function that loads a page and waits for timeout
        input:
            link - Link to the page we wish to load
            timeout - Seconds to wait before timing out
            wait_after - Seconds to wait after loading
        output:
            N/A
        """

        # Visit link
        self.driver.get(link)
        wait = WebDriverWait(self.driver, timeout)

        # Attempt to open link
        try:
            if athlete_page:
                wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@class='athlete-info left-side']")))
            else:
                wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@class='uk-container']")))
        except TimeoutException:
            print("Timed out waiting for page " + link + " to load")
            self.driver.quit()

        # Wait for page to load
        time.sleep(wait_after)
        
    def get_comp_years_and_league(self):
        """
        Parse the world-competition/last-result page to find and return years and leagues
        input:
            N/A
        output:
            List of tuples containing comp year and league
        """
        year_league_comb = []
        
        try:
            self.load_page(IFSCScraper.url)
        except:
            print('Error loading page!')
            
        # The dropdown menus to pick years/leagues/events is in an iframe - we need to switch to it
        frame = self.driver.find_element(By.XPATH, '/html/body/div[1]/div[4]/div/iframe')
        self.driver.switch_to.frame(frame)

        # Dropdown menus for each choice
        year_dd     = self.driver.find_element(By.XPATH, '//select[@id="years"]')
        league_dd   = self.driver.find_element(By.XPATH, '//select[@id="indexes"]')

        # Select all options for 'Year' and 'League' dropdown menus
        year_opts = Select(year_dd).options
        league_opts = Select(league_dd).options

        # Extract text of each of the above options
        years   = [opt.text for opt in year_opts]
        # leagues = [opt.text for opt in league_opts[1:]] # all leagues
        league = league_opts[1].text #world cup only for now

        year_league_comb = []
        for year in years:
            if (year, league) not in year_league_comb:
                year_league_comb.append((year, league))

        return year_league_comb

    def get_years_events(self, years):
        """
        Iterate through each year and get that years events
        input:
            N/A
        output:
            List of tuples containing comp year and league
        """
        try:
            self.load_page(IFSCScraper.url)
        except:
            print('Error loading page!')
        
        # The dropdown menus to pick years/leagues/events is in an iframe - we need to switch to it
        frame = self.driver.find_element(By.XPATH, '/html/body/div[1]/div[4]/div/iframe')
        self.driver.switch_to.frame(frame)

        # Dropdown menus for each choice
        year_dd, league_dd, event_dd, cat_dd = self.get_dropdowns(self.driver)
                
        # Creating wait
        wait = WebDriverWait(self.driver, 10)
        
        # Iterate through years provided by get_comp_years_and_league
        for year in years:                
            years_ob   = Select(year_dd).select_by_value(year[0])   #0 is most recent year (2023)
            leagues_ob = Select(league_dd).select_by_index(1) #starts at index 1
        
            # IF THINGS BREAK, UNCOMMENT THIS SECTION
            # Selects third dropdown menu, events
            # event_dd = self.driver.find_element(By.XPATH, '//select[@id="events"]')
            
            # Waits for third dropdown to populate with options based on leagues_ob                                        
            events_select = Select(event_dd)
            wait.until(lambda d: len(events_select.options) > 1)
            
            # Gets event options and ids for year and add to dictionary
            event_opts = Select(event_dd).options
            events = [opt.text for opt in event_opts[1:]]
            event_ids = [opt.get_attribute("value").split('/')[-1] for opt in event_opts[1:]]
            self.year_events[year[0]] = [(event, id) for event, id in zip(events, event_ids)]
            
            print(f'{year[0]}: {self.check_for_event_results(self.driver, event_dd, events)}')
    
    def check_for_event_results(self, driver, dd, events):
        """
        Iterate through each event and check if results exist
        input:
            N/A
        output:
            unsure yet
        """
        event_has_results = []
        for event in events:
            # Select event in events dropdown
            events_ob = Select(dd).select_by_visible_text(event)

            # Select fourth dropdown menu, categories
            cat_dd = driver.find_element(By.XPATH, '//select[@id="categories"]')
            cat_select = Select(cat_dd)
            wait = WebDriverWait(driver, 2)
            try:
                wait.until(lambda d: len(cat_select.options) > 1)
                event_has_results.append(1)
            except:
                event_has_results.append(0)
        return f'{sum(event_has_results)} of {len(events)} events have results!'
    
    def get_year_list(self):
        try:
            self.load_page(IFSCScraper.url)            
            wait = WebDriverWait(self.driver, 5)
            
            # The data we are after resides within an iframe
            wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe.jch-lazyloaded")))
        except:
            print('Error loading page!')
            
        # Dropdown menus for each choice
        year_dd, league_dd, event_dd, cat_dd = self.get_dropdowns(self.driver)
        year_opts = Select(year_dd).options
        return [year.text for year in year_opts]
    
    def get_single_year(self, year = '2022'):
        """
        Fully scrape each event for a given year
        input:
            year (string) - year to be scraped
        output:
            List of tuples (category, title, date, dataframe) to 
            be passed into function that generates actual .csv file
        """
        try:
            self.load_page(IFSCScraper.url)            
            wait = WebDriverWait(self.driver, 8)
            
            # The data we are after resides within an iframe
            wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe.jch-lazyloaded")))
            print(f'Scraping {year}...')
        except:
            print('Error loading page!')        
        
        # Dropdown menus for each choice
        year_dd, league_dd, event_dd, cat_dd = self.get_dropdowns(self.driver)
        
        # Select given year and league
        year_ob   = Select(year_dd).select_by_visible_text(year)
        league_ob = Select(league_dd).select_by_index(1)
        
        # Get list of all events for the year
        all_events = self.get_events(self.driver, event_dd)
        
        # Loop through each event, scrape results, and generate .csv file
        dfs = []
        for i, event in enumerate(all_events):
            # Implement check to see if event has already been scraped
            if self.check_if_scraped(event) and all_events[i] != all_events[i-1]:
                print(f'--Already scraped {event}!')
                continue
            # Some of the events aren't actually events, but more qualification rounds, and 
            # they don't list the results correctly, which will cause errors. The common thread
            # is the naming of them.
            elif event.count('(') < 1:
                print(f'--Skipping {event}...')
                continue
            else:
                # Set this flag for special cases where two events share the same name (rare)
                same_event_name = True if all_events[i] == all_events[i-1] else False
                                    
                # Select event
                if same_event_name:
                    event_ob = Select(event_dd).select_by_index(i+1)
                else:
                    event_ob = Select(event_dd).select_by_visible_text(event)
                    
                category_select = Select(cat_dd)

                # Some events were cancelled or don't have results listed, check for it here
                try:
                    wait.until(lambda d: len(category_select.options) > 1)
                    if not self.check_if_scraped(event):
                        # self.add_to_scraped_file(event)
                        print(f'--Scraping {event}...')
                except:
                    print(f'--No data for {event}!')
                    continue

                # Get results for each category
                for cat in category_select.options[1:]:
                    cat_ob = Select(cat_dd).select_by_visible_text(cat.text) # selects category

                    # Finds table with desired data
                    try:
                        wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@id="table_id_wrapper"]')))
                    except:
                        print(f'----No data for {cat.text}!')
                        continue

                    table_wrapper = self.driver.find_element(By.XPATH, '//div[@id="table_id_wrapper"]')
                    results = table_wrapper.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')

                    # Get event name and date
                    event_details = self.driver.find_element(By.XPATH, '//div[@class="labels"]')
                    event_results = event_details.find_elements(By.TAG_NAME, 'p') # Event title & date

                    # Get filename to check if it exists already
                    file = self.generate_filename((cat.text, event_results[0].text, event_results[1].text))
                    text = '--' + file
                    path = self.get_dir(cat.text)
                    
                    if same_event_name:
                        text = text[:-4] + '2' + '.csv'
                        file = file[:-4] + '2' + '.csv'
                    
                    filepath = os.path.join(path, file)
                                        
                    # Checks if the filename has been added to the .txt, AND if the file exists
                    if self.check_if_scraped(text, filepath):
                        continue
                    else:
                        print(f'----Scraping {cat.text}...')
                        
                        # Data (list of dictionaries) contains each climber's results
                        data = []
                        for result in results:
                            # Each climber's result stored in dict
                            temp_dict = self.scrape_results(event, result, cat.text)
                            
                            if temp_dict:
                                data.append(temp_dict)
                            else:
                                print(f'----Data format error for {cat.text}!')

                        # Create dataframe after collecting all the data
                        df = pd.DataFrame.from_dict(data)
                        
                        # Convert raw results into a .csv and marks file as scraped
                        self.convert_to_csv(file, cat.text, df)
                        self.add_to_scraped_file('--' + file)
                        
                # All categories for the event have been scraped
                self.add_to_scraped_file(event)
                        
    def get_athlete_height(self, athlete_id):
        """
        Scrape athlete page for climber's height
        input:
            athlete_id (int) - unique id assigned to each climber
        output:
            Height (in cm)
        """
        try:
            self.load_page(IFSCScraper.url2 + athlete_id, athlete_page=1)            
        except:
            print('Error loading page!')
            self.driver.quit()
        
        height_div = self.driver.find_element(By.XPATH, '//div[@class="athlete-info left-side"]')
        height_ele = height_div.find_elements(By.TAG_NAME, 'div')[1]
        return height_ele.text.split(': ')[1]
    
    def add_to_scraped_file(self, text):
        if not self.check_if_scraped(text):
            with open(ALREADY_SCRAPED, 'a') as file:
                file.write(f'{text}\n')
                return
    
    def check_if_scraped(self, text, file = ''):
        with open(ALREADY_SCRAPED, 'r') as f:
            done = [x.strip() for x in f.readlines()]
            
        if file:
            if text in done and os.path.exists(file):
                return True
            return False
        return text in done

    def scrape_results(self, event, result, cat):
        details = result.find_elements(By.TAG_NAME, 'td')
        athlete_id = details[1].find_element(By.TAG_NAME, 'a').get_attribute('href').split('id=')[1]
        
        if "LEAD" in cat or "BOULDER" in cat:
            try:
                temp_dict = {
                    "Event": event,
                    "ID": athlete_id,
                    "Rank": details[0].text,
                    "Name": f"{details[1].text} {details[2].text}",
                    "Country": details[3].text,
                    "Qualification": details[4].text,
                    "Semi-Final": details[5].text,
                    "Final": details[6].text
                }
            except:
                return False
        elif "SPEED" in cat:
            try:
                temp_dict = {
                    "Event": event,
                    "ID": athlete_id,
                    "Rank": details[0].text,
                    "Name": f"{details[1].text} {details[2].text}",
                    "Country": details[3].text,
                    "Qualification": details[4].text,
                    "Final": details[5].text
                }
            except:
                return False
        else:
            try:
                temp_dict = {
                    "Event": event,
                    "ID": athlete_id,
                    "Rank": details[0].text,
                    "Name": f"{details[1].text} {details[2].text}",
                    "Country": details[3].text,
                    "Qualification": details[4].text
                }
            except:
                return False
        return temp_dict
    
    def get_dropdowns(self, driver):
        year_dd   = driver.find_element(By.XPATH, '//select[@id="years"]')
        league_dd = driver.find_element(By.XPATH, '//select[@id="indexes"]')
        event_dd  = driver.find_element(By.XPATH, '//select[@id="events"]')
        cat_dd    = driver.find_element(By.XPATH, '//select[@id="categories"]')        
        return year_dd, league_dd, event_dd, cat_dd
    
    def get_events(self, driver, events_dd):
        event_opts = Select(events_dd)
        wait = WebDriverWait(driver, 10)
        wait.until(lambda d: len(event_opts.options) > 1)                    
        return [x.text for x in event_opts.options[1:]]
    
    def generate_filename(self, packed_data):
        # Unpacks data
        (category, event, date) = packed_data

        # Create filename in form of {date}_{event}_{category}
        date = ' '.join(date.split()[::-1][:2])       

        # Cleans up event name for next part
        event = event.replace('- ', '').split()
        if event[-1] == 'CANCELLED':
            event = ' '.join(event[:-2])
        else:
            event = ' '.join(event[:-1])

        # Uses Regex to clean because not every name has the same format
        filename = ' '.join([date, event, category])
        filename = re.findall("^[^\(]+|[\(].*", filename)
        filename[1] = filename[1].split(') ', 1)[1]
        filename = (''.join(filename)
                    .replace('(','[')
                    .replace(')',']')
                    .replace(' ', '_')
                    .replace(',', '')
                    .lower()) + '.csv'        
        return filename

    def convert_to_csv(self, filename, category, data):
        # Figure out correct directory
        path = self.get_dir(category)
        file = path + f'\\{filename}'

        # Generates .csv with filename
        data.to_csv(file, index=False)
            
    def get_dir(self, category):
        base = category.upper().split()
        if "MEN" in base:
            if "BOULDER" in base: return BOULDER_MEN_DIR
            if "LEAD" in base: return LEAD_MEN_DIR
            if "SPEED" in base: return SPEED_MEN_DIR
            if "COMBINED" in base: return COMBINED_MEN_DIR
            if "BOULDER&LEAD" in base: return BOULDER_AND_LEAD_MEN_DIR
        if "WOMEN" in base:
            if "BOULDER" in base: return BOULDER_WOMEN_DIR
            if "LEAD" in base: return LEAD_WOMEN_DIR
            if "SPEED" in base: return SPEED_WOMEN_DIR
            if "COMBINED" in base: return COMBINED_WOMEN_DIR
            if "BOULDER&LEAD" in base: return BOULDER_AND_LEAD_WOMEN_DIR
                
    def end_session(self):
        self.driver.quit()
    
    def scrape_site(self):
        self.get_years_events(self.get_comp_years_and_league())
        self.end_session()
        
    def scrape_all_ifsc_world_cups(self):
        years = self.get_year_list()
        for year in years[1:17]: # 2022-2007, 2023 events haven't happened yet
            scraper.get_single_year(year)
        self.end_session()

In [4]:
# Takes ~37 minutes to scrape all years and events
# scraper = IFSCScraper()
# scraper.scrape_all_ifsc_world_cups()
# scraper.get_single_year('2022')
# scraper.end_session()

# Analysis

## Helper Functions

In [5]:
def clean_boulder(df):
    # Gets tops and zones for qualifications
    df[['Q_Top', 'Q_Zone']] = df['Qualification'].str.split('T', expand=True)
    df['Q_Zone'] = df['Q_Zone'].str.split('Z', expand=True)[0]
    df[['Qualification', 'Q_Top_Att', 'Q_Zone_Att']] = df['Qualification'].str.split(expand=True)

    # Gets tops and zones for semi-finals
    df[['S_Top', 'S_Zone']] = df['Semi-Final'].str.split('T', expand=True)
    df['S_Zone'] = df['S_Zone'].str.split('Z', expand=True)[0]
    df[['Semi-Final', 'S_Top_Att', 'S_Zone_Att']] = df['Semi-Final'].str.split(expand=True)

    # Gets tops and zones for finals
    df[['F_Top', 'F_Zone']] = df['Final'].str.split('T', expand=True)
    df['F_Zone'] = df['F_Zone'].str.split('Z', expand=True)[0]
    df[['Final', 'F_Top_Att', 'F_Zone_Att']] = df['Final'].str.split(expand=True)

    new_cols = ['Q_Top', 'Q_Zone', 'Q_Top_Att', 'Q_Zone_Att',
                'S_Top', 'S_Zone', 'S_Top_Att', 'S_Zone_Att',
                'F_Top', 'F_Zone', 'F_Top_Att', 'F_Zone_Att']

    # Convert all new columns to int
    for col in new_cols:
        df[col] = df[col].astype('float', errors = 'ignore')

    return df

# Function to correct the old scoring syntax
def convert_score(score):
    if score:
        tmp = str(score)
        if 'b' in tmp.lower():
            tops, top_att = tmp.lower().split()[0].split('t')
            zones, zone_att = tmp.lower().split()[1].split('b')

            new_score = tops + 'T' + zones + 'Z ' + top_att + ' ' + zone_att
            return new_score
        return score
    return score

# Function to display a single climber's results
def check_climber(df, name):
    name = name.upper()
    individual = df.groupby(df['Name']).get_group(name)
    return individual

# Function to show the count of a climber's round appearances
def count_round_appearances(df, name):
    name = name.upper()
    # Gets number of time climber appeared for each round
    q = df.groupby(df['Name']).get_group(name).count()['Qualification']
    s = df.groupby(df['Name']).get_group(name).count()['Semi-Final']
    f = df.groupby(df['Name']).get_group(name).count()['Final']
    
    print(f'{name} appearances:')
    print(f'{"Qualification round:" :>20} {q :>3}')
    print(f'{"Semi-Final round:" :>20} {s :>3}')
    print(f'{"Final round:" :>20} {f :>3}')
    print(f'{"Toal appearances:" :>20} {q+s+f :>3}')
    
def get_bouldering_results(category = BOULDER_MEN_DIR):
    # Grabs all .csv files in given directory and combine into dataframe
    all_files = glob.glob(os.path.join(BOULDER_MEN_DIR, "*.csv"))
    df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

    # Clean up some column/datatype issues
    df['Rank'] = df['Rank'].fillna(-1).astype('int') # Fixes DNS rank
    df['Qualification'] = df['Qualification'].apply(convert_score) # Fixes old scoring syntax
    df['Semi-Final'] = df['Semi-Final'].apply(convert_score)
    df['Final'] = df['Final'].apply(convert_score)
    return df

In [21]:
boulder_men_all_results = get_bouldering_results()
count_round_appearances(boulder_men_all_results, 'jongwon chon')
# check_climber(boulder_men, 'jongwon chon')

JONGWON CHON appearances:
Qualification round:  46
   Semi-Final round:  41
        Final round:  25
   Toal appearances: 112


## Climber Statistics

In [53]:
def get_bouldering_stats(df, category='men'):
    # Checks if we've already created this file
    if category.lower() == 'men':
        csv = 'bo_stats_men.csv'
    else:
        csv = 'climber_stats_women.csv'
    
    file = DATA_DIR + '\\' + csv
        
    if os.path.exists(file):
        print(f'Loading {csv}...')
        return pd.read_csv(file, index_col = 0)
    
    print('File does not exist - creating it now...')
    
    # Create a new dataframe with athlete's ID, name, and country
    clean_df = df.drop_duplicates(subset=['Name', 'ID'])[['ID', 'Name', 'Country']]

    # Counts number of appearances per round per climber
    qual_all = df.groupby(['ID', 'Name'])['Qualification'].count()
    semi_app = df.groupby(['ID', 'Name'])['Semi-Final'].count()
    final_app = df.groupby(['ID', 'Name'])['Final'].count()

    # Merges the above dataframes
    rounds = pd.merge(pd.merge(qual_all, semi_app , left_on=['ID', 'Name'], right_index=True), final_app, left_on=['ID', 'Name'], right_index=True)
    clean_df = pd.merge(clean_df, rounds, left_on=['ID', 'Name'], right_index=True)

    # Calculates percentages of appearances in each round
    clean_df['Q_Pct'] = round((clean_df['Qualification'] / clean_df['Qualification']) * 100, 2)
    clean_df['S_Pct'] = round((clean_df['Semi-Final'] / clean_df['Qualification']) * 100, 2)
    clean_df['F_Pct'] = round((clean_df['Final'] / clean_df['Qualification']) * 100, 2)
    clean_df = clean_df.sort_values('Final', ascending=False).reset_index(drop=True)

    # Convert 'ID' to string so we can scrape site for heights (~33 minutes)
    clean_df['ID'] = clean_df['ID'].apply(str)
    clean_df['Height'] = clean_df['ID'].apply(lambda x: scraper.get_athlete_height(x))

    # Saves data so we don't have to scrape later    
    clean_df.to_csv(file)
    print('...Done!')
    
    return clean_df

In [54]:
boulder_men_stats = get_bouldering_stats(boulder_men_all_results)
boulder_men_stats

Loading climber_stats_men.csv...


,ID,Name,Country,Qualification,Semi-Final,Final,Q_Pct,S_Pct,F_Pct,Height
0,1204,KILIAN FISCHHUBER,AUT,56,51,44,100.0,91.07,78.57,-
1,53,DMITRII SHARAFUTDINOV,RUS,65,53,40,100.0,81.54,61.54,-
2,60,RUSTAM GELMANOV,RUS,69,56,38,100.0,81.16,55.07,-
3,2272,KOKORO FUJII,JPN,54,48,27,100.0,88.89,50.00,176
4,79,ALEKSEY RUBTSOV,RUS,57,50,26,100.0,87.72,45.61,178
...,...,...,...,...,...,...,...,...,...,...
1297,6063,DAVID WETMORE,USA,2,0,0,100.0,0.00,0.00,-
1298,1736,AUSTIN GEIMAN,USA,15,1,0,100.0,6.67,0.00,-
1299,6081,WOLTER WESTDIJK,NED,1,0,0,100.0,0.00,0.00,-
1300,6080,MELLE VROOM,NED,1,0,0,100.0,0.00,0.00,-


In [55]:
# Number of climbers that have a height listed on the IFSC site
have_height = (~boulder_men_stats['Height'].str.contains('-')).sum()
no_height = (boulder_men_stats['Height'].str.contains('-')).sum()
print(f'Only {have_height} climbers ({(have_height/no_height)*100:.2f}%) have their height listed!')

Only 153 climbers (13.32%) have their height listed!


## Country Statistics

In [56]:
def get_bouldering_country_stats(df):
    clean_df = pd.DataFrame()
    clean_df['Country'] = df['Country'].unique()

    # Counts number of appearances per round per country
    qual_all = df.groupby(['Country'])['Qualification'].count()
    semi_app = df.groupby(['Country'])['Semi-Final'].count()
    final_app = df.groupby(['Country'])['Final'].count()

    # Merges the above with our previous dataframe
    clean_df = pd.merge(pd.merge(qual_all, semi_app , left_on='Country', right_index=True),
                  final_app, left_on='Country', right_index=True)

    # Calculates percentages of appearances vs total events
    clean_df = clean_df.sort_values('Final', ascending=False).reset_index()
    clean_df['Q_Pct'] = round((clean_df['Qualification'] / clean_df['Qualification'].sum()) * 100, 2)
    clean_df['S_Pct'] = round((clean_df['Semi-Final'] / clean_df['Semi-Final'].sum()) * 100, 2)
    clean_df['F_Pct'] = round((clean_df['Final'] / clean_df['Final'].sum()) * 100, 2)
    return clean_df

In [57]:
boulder_men_country_stats = get_bouldering_country_stats(boulder_men_all_results)
boulder_men_country_stats

,Country,Qualification,Semi-Final,Final,Q_Pct,S_Pct,F_Pct
0,JPN,651,386,134,8.36,18.15,20.43
1,RUS,425,197,109,5.46,9.26,16.62
2,FRA,628,299,81,8.06,14.06,12.35
3,AUT,390,142,75,5.01,6.68,11.43
4,GER,495,180,38,6.35,8.46,5.79
...,...,...,...,...,...,...,...
75,IRL,33,1,0,0.42,0.05,0.00
76,ARG,33,0,0,0.42,0.00,0.00
77,KAZ,32,0,0,0.41,0.00,0.00
78,KGZ,1,0,0,0.01,0.00,0.00


In [ ]:
# df[df['Country'] == 'JPN']['Name'].unique()

In [ ]:
# cols = ['Qualification', 'Semi-Final', 'Final', 'Q_Pct', 'S_Pct', 'F_Pct']
# for col in cols:
#     print(country_appearances[col].sum())